In [13]:
from SBM_SDE import *
from obs_and_flow_classes_and_functions import *
from get_CO2 import *
import seaborn as sns
import torch
from torch import nn
import torch.distributions as d
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import math
from tqdm import tqdm
import random
from torch.autograd import Function
import argparse
import os
import sys
from pathlib import Path
import shutil
import pandas as pd

In [14]:
torch.manual_seed(2)
devi = torch.device("".join(["cuda:",f'{cuda_id}']) if torch.cuda.is_available() else "cpu")

cuda_id = 1
dt = .2 #SDE discretization timestep.
t = 150 #Simulation run for T hours.
n = int(t / dt) 
t_span = np.linspace(0, t, n + 1)
t_span_tensor = torch.reshape(torch.Tensor(t_span), [1, n + 1, 1]) #T_span needs to be converted to tensor object. Additionally, facilitates conversion of I_S and I_D to tensor objects.
l_r = 1e-3
niter = 1500
piter = 100
batch_size = 2 #Number of sets of observation outputs to sample per set of parameters.
state_dim_SCON = 3 #Not including CO2 in STATE_DIM, because CO2 is an observation.
state_dim_SAWB = 4 #Not including CO2 in STATE_DIM, because CO2 is an observation.

In [15]:
obs_times, obs_means_CON, obs_error_CON = csv_to_obs_df('CON_synthetic_sol_df.csv', state_dim_SCON + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_CON)
print(obs_error_CON)

tensor([[4.5660e+01, 4.5647e+01, 4.5650e+01, 4.5643e+01, 4.5638e+01, 4.5640e+01,
         4.5626e+01, 4.5630e+01, 4.5618e+01, 4.5617e+01, 4.5615e+01, 4.5605e+01,
         4.5609e+01, 4.5595e+01, 4.5598e+01, 4.5591e+01],
        [7.1469e-02, 7.3700e-02, 7.3289e-02, 7.4423e-02, 7.5230e-02, 7.4875e-02,
         7.6935e-02, 7.6316e-02, 7.8034e-02, 7.7924e-02, 7.8087e-02, 7.9339e-02,
         7.8534e-02, 8.0440e-02, 7.9792e-02, 8.0558e-02],
        [7.1469e-01, 7.1372e-01, 7.1409e-01, 7.1363e-01, 7.1332e-01, 7.1362e-01,
         7.1266e-01, 7.1317e-01, 7.1240e-01, 7.1253e-01, 7.1251e-01, 7.1189e-01,
         7.1246e-01, 7.1149e-01, 7.1193e-01, 7.1158e-01],
        [1.1000e-03, 1.8381e-03, 4.6086e-04, 3.0443e-03, 4.7108e-04, 1.8966e-03,
         1.1670e-03, 7.0596e-04, 2.7906e-03, 4.2941e-04, 2.8298e-03, 7.3398e-04,
         1.2239e-03, 2.0338e-03, 5.1562e-04, 3.3495e-03]])
tensor([[4.5624e+00, 7.6809e-03, 7.1285e-02, 1.5369e-04]])


In [16]:
obs_times, obs_means_AWB, obs_error_AWB = csv_to_obs_df('AWB_synthetic_sol_df.csv', state_dim_SAWB + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_AWB)
print(obs_error_AWB)

tensor([[5.3602e+01, 5.3588e+01, 5.3591e+01, 5.3584e+01, 5.3578e+01, 5.3579e+01,
         5.3564e+01, 5.3569e+01, 5.3560e+01, 5.3559e+01, 5.3558e+01, 5.3545e+01,
         5.3551e+01, 5.3536e+01, 5.3538e+01, 5.3531e+01],
        [1.1421e-01, 1.1832e-01, 1.1810e-01, 1.1999e-01, 1.2125e-01, 1.2086e-01,
         1.2401e-01, 1.2294e-01, 1.2498e-01, 1.2491e-01, 1.2482e-01, 1.2692e-01,
         1.2544e-01, 1.2826e-01, 1.2713e-01, 1.2800e-01],
        [6.7073e-01, 6.7239e-01, 6.7187e-01, 6.7274e-01, 6.7348e-01, 6.7354e-01,
         6.7549e-01, 6.7470e-01, 6.7601e-01, 6.7623e-01, 6.7639e-01, 6.7818e-01,
         6.7750e-01, 6.7964e-01, 6.7946e-01, 6.8049e-01],
        [1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3416e-02,
         1.3416e-02, 1.3416e-02, 1.3417e-02, 1.3417e-02, 1.3418e-02, 1.3419e-02,
         1.3419e-02, 1.3420e-02, 1.3421e-02, 1.3422e-02],
        [1.1000e-03, 1.6757e-03, 5.7473e-04, 2.4797e-03, 5.9680e-04, 1.7478e-03,
         1.2328e-03, 8.2910e-04, 2.4061

In [17]:
obs_times, obs_means_AWB_ECA, obs_error_AWB_ECA = csv_to_obs_df('AWB_ECA_synthetic_sol_df.csv', state_dim_SAWB + 1, t, 0.1) #Need to +1 because data has CO2 observations on top of other state observations.
print(obs_means_AWB_ECA)
print(obs_error_AWB_ECA)

tensor([[5.3606e+01, 5.3592e+01, 5.3594e+01, 5.3586e+01, 5.3579e+01, 5.3581e+01,
         5.3567e+01, 5.3572e+01, 5.3559e+01, 5.3558e+01, 5.3555e+01, 5.3545e+01,
         5.3550e+01, 5.3533e+01, 5.3538e+01, 5.3528e+01],
        [1.9081e-01, 1.9506e-01, 1.9527e-01, 1.9779e-01, 1.9962e-01, 1.9968e-01,
         2.0286e-01, 2.0196e-01, 2.0493e-01, 2.0501e-01, 2.0569e-01, 2.0746e-01,
         2.0642e-01, 2.0993e-01, 2.0873e-01, 2.1039e-01],
        [6.7073e-01, 6.7236e-01, 6.7196e-01, 6.7284e-01, 6.7358e-01, 6.7335e-01,
         6.7505e-01, 6.7448e-01, 6.7616e-01, 6.7630e-01, 6.7668e-01, 6.7806e-01,
         6.7749e-01, 6.7978e-01, 6.7921e-01, 6.8067e-01],
        [1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3415e-02, 1.3416e-02,
         1.3416e-02, 1.3416e-02, 1.3417e-02, 1.3417e-02, 1.3418e-02, 1.3418e-02,
         1.3419e-02, 1.3420e-02, 1.3421e-02, 1.3422e-02],
        [1.1000e-03, 1.6543e-03, 5.6910e-04, 2.4475e-03, 5.8824e-04, 1.7276e-03,
         1.2063e-03, 8.1481e-04, 2.3599

In [18]:
temp_ref = 283

#System parameters from deterministic CON model
u_M = 0.002
a_SD = 0.33
a_DS = 0.33
a_M = 0.33
a_MSC = 0.5
k_S_ref = 0.000025
k_D_ref = 0.005
k_M_ref = 0.0002
Ea_S = 75
Ea_D = 50
Ea_M = 50

#SCON diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1

SCON_C_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC}
SCON_SS_params_dict = {'u_M': u_M, 'a_SD': a_SD, 'a_DS': a_DS, 'a_M': a_M, 'a_MSC': a_MSC, 'k_S_ref': k_S_ref, 'k_D_ref': k_D_ref, 'k_M_ref': k_M_ref, 'Ea_S': Ea_S, 'Ea_D': Ea_D, 'Ea_M': Ea_M, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC}

In [19]:
#System parameters from deterministic AWB model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_D = 200
K_U = 1
V_D_ref = 0.4
V_U_ref = 0.02
Ea_V_D = 75
Ea_V_U = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#SAWB diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_D': K_D, 'K_U': K_U, 'V_D_ref': V_D_ref, 'V_U_ref': V_U_ref, 'Ea_V_D': Ea_V_D, 'Ea_V_U': Ea_V_U, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [20]:
#System parameters from deterministic model
u_Q_ref = 0.2
Q = 0.002
a_MSA = 0.5
K_DE = 200
K_UE = 1
V_DE_ref = 0.4
V_UE_ref = 0.02
Ea_V_DE = 75
Ea_V_UE = 50
r_M = 0.0004
r_E = 0.00001
r_L = 0.0005

#Diffusion matrix sigma scale parameters
c_SOC = 1.
c_DOC = 0.01
c_MBC = 0.1
c_EEC = 0.001
s_SOC = 0.1
s_DOC = 0.1
s_MBC = 0.1
s_EEC = 0.1

SAWB_ECA_C_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 'c_SOC': c_SOC, 'c_DOC': c_DOC, 'c_MBC': c_MBC, 'c_EEC': c_EEC}
SAWB_ECA_SS_params_dict = {'u_Q_ref': u_Q_ref, 'Q': Q, 'a_MSA': a_MSA, 'K_DE': K_DE, 'K_UE': K_UE, 'V_DE_ref': V_DE_ref, 'V_UE_ref': V_UE_ref, 'Ea_V_DE': Ea_V_DE, 'Ea_V_UE': Ea_V_UE, 'r_M': r_M, 'r_E': r_E, 'r_L': r_L, 's_SOC': s_SOC, 's_DOC': s_DOC, 's_MBC': s_MBC, 's_EEC': s_EEC}

In [21]:
#Obtain SOC and DOC pool litter inputs for all SBMs.
i_s_tensor = 0.001 + 0.0005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous SOC input function
i_d_tensor = 0.0001 + 0.00005 * torch.sin((2 * np.pi / (24 * 365)) * t_span_tensor) #Exogenous DOC input function

In [22]:
def neg_log_lik(C_PATH, T_SPAN_TENSOR, DT, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF):
    drift, diffusion_sqrt = DRIFT_DIFFUSION(C_PATH[:, :-1, :], T_SPAN_TENSOR[:, :-1, :], I_S_TENSOR[:, :-1, :], I_D_TENSOR[:, :-1, :], PARAMS_DICT, TEMP_GEN, TEMP_REF)
    euler_maruyama_state_sample_object = d.multivariate_normal.MultivariateNormal(loc = C_PATH[:, :-1, :] + drift * DT, scale_tril = diffusion_sqrt * math.sqrt(DT))
    return -euler_maruyama_state_sample_object.log_prob(C_PATH[:, 1:, :]).sum(-1)

In [23]:
def train(DEVICE, L_R, NITER, PRETRAIN_ITER, BATCH_SIZE, ObsModel, csv_to_obs_df, DATA_CSV, OBS_ERROR_SCALE, STATE_DIM, T, DT, N, T_SPAN_TENSOR, I_S_TENSOR, I_D_TENSOR, DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF, ANALYTICAL_STEADY_STATE_INIT, GET_CO2):
    obs_times, obs_means, obs_error = csv_to_obs_df(DATA_CSV, STATE_DIM + 1, T, OBS_ERROR_SCALE) #Need to +1 because data has CO2 observations on top of other state observations. 
    obs_model_no_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means[:-1, :], obs_error[:, :-1]) #Hack for bypassing ObsModel and SDEFlow dimension mismatch issue.
    obs_model_CO2 = ObsModel(DEVICE, obs_times, DT, obs_means, obs_error)
    net = SDEFlow(DEVICE, BATCH_SIZE, obs_model_no_CO2, STATE_DIM, T, DT, N).to(DEVICE)
    optimizer = optim.Adam(net.parameters(), lr = L_R) 
    if PRETRAIN_ITER >= NITER:
        raise Exception("PRETRAIN_ITER must be < NITER.")
    best_loss_norm = 1e10
    best_loss_ELBO = 1e20
    norm_losses = [best_loss_norm] * 10
    ELBO_losses = [best_loss_ELBO] * 10
    C0 = ANALYTICAL_STEADY_STATE_INIT(I_S_TENSOR[0, 0, 0].item(), I_D_TENSOR[0, 0, 0].item(), PARAMS_DICT) #Calculate deterministic initial conditions.
    C0 = C0[(None,) * 2].repeat(BATCH_SIZE, 1, 1).to(DEVICE) #Assign initial conditions to C_PATH.
    with tqdm(total = NITER, desc = f'Train Diffusion', position = -1) as tq:
        for iter in range(NITER):
            net.train()
            optimizer.zero_grad()
            C_PATH, log_prob = net() #Obtain paths with solutions at times after t0.
            C_PATH = torch.cat([C0, C_PATH], 1) #Append deterministic CON initial conditions conditional on parameter values to C path. 
            CO2 = GET_CO2(C_PATH, T_SPAN_TENSOR, PARAMS_DICT, TEMP_GEN, TEMP_REF)
            x_with_CO2 = torch.cat([C_PATH, CO2], -1)
            if iter <= PRETRAIN_ITER:
                #l1_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                #l1_norm = torch.sum(torch.abs(l1_norm_element)).mean()
                #best_loss_norm = l1_norm if l1_norm < best_loss_norm else best_loss_norm
                #norm_losses.append(l1_norm.item())
                l2_norm_element = x_with_CO2 - torch.mean(obs_model_CO2.mu, -1)
                l2_norm = torch.sqrt(torch.sum(torch.square(l2_norm_element))).mean()
                best_loss_norm = l2_norm if l2_norm < best_loss_norm else best_loss_norm
                norm_losses.append(l2_norm.item())
                if len(norm_losses) > 10:
                    norm_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average norm loss at {iter} iterations is: {sum(norm_losses) / len(norm_losses)}. Best norm loss value is: {best_loss_norm}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                #l1_norm.backward()
                l2_norm.backward()
            else:
                log_lik = neg_log_lik(C_PATH, T_SPAN_TENSOR.to(DEVICE), dt, I_S_TENSOR.to(DEVICE), I_D_TENSOR.to(DEVICE), DRIFT_DIFFUSION, PARAMS_DICT, TEMP_GEN, TEMP_REF)
                ELBO = log_prob.mean() + log_lik.mean() - obs_model_CO2(x_with_CO2) #obs_model_CO2(x_with_CO2) is obs log likelihood.
                best_loss_ELBO = ELBO if ELBO < best_loss_ELBO else best_loss_ELBO
                ELBO_losses.append(ELBO.item())
                if len(ELBO_losses) > 10:
                    ELBO_losses.pop(0)
                if iter % 10 == 0:
                    print(f"Moving average ELBO loss at {iter} iterations is: {sum(ELBO_losses) / len(ELBO_losses)}. Best ELBO loss value is: {best_loss_ELBO}.")
                    print('\nx with CO2 means across time =', x_with_CO2.mean(-2))
                    print('\nx with CO2 =', x_with_CO2)
                ELBO.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 3.0)
            optimizer.step()
            if iter % 100000 == 0 and iter > 0:
                optimizer.param_groups[0]['lr'] *= 0.1
            tq.update()

In [ ]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_C, SCON_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)


Train Diffusion:   0%|          | 0/1500 [00:00<?, ?it/s]

Moving average norm loss at 0 iterations is: 9000000173.503454. Best norm loss value is: 1735.0345458984375.

x with CO2 means across time = tensor([[0.9018, 0.7048, 0.6691, 0.0029],
        [0.8806, 0.7082, 0.6642, 0.0030]], grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.0500e-01, 9.6841e-01, 6.6545e-01, 3.4796e-03],
         [6.3360e-01, 5.9831e-01, 6.7865e-01, 2.2781e-03],
         ...,
         [6.9264e-01, 6.2056e-01, 5.5705e-01, 4.8133e-03],
         [6.9130e-01, 5.4380e-01, 6.3485e-01, 4.2789e-03],
         [1.0544e+00, 9.8098e-01, 8.4000e-01, 7.6251e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [7.0500e-01, 7.1802e-01, 5.3653e-01, 2.5892e-03],
         [7.3629e-01, 9.0377e-01, 8.6091e-01, 3.4132e-03],
         ...,
         [2.2301e+00, 8.0130e-01, 4.7395e-01, 6.2161e-03],
         [7.8987e-01, 7.1217e-01, 8.7588e-01, 5.6105e-03],
         [4.5167e-01, 7.3495e-01, 3.9585e-01, 5.6243e-03


Train Diffusion:   1%|          | 10/1500 [00:06<14:56,  1.66it/s]

Moving average norm loss at 10 iterations is: 1638.4563354492188. Best norm loss value is: 1579.98974609375.

x with CO2 means across time = tensor([[5.1876e+00, 4.0139e-01, 1.7050e-01, 1.7593e-03],
        [5.0817e+00, 3.9070e-01, 1.7059e-01, 1.7160e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [1.1752e+00, 5.1720e+00, 2.5150e-01, 1.8077e-02],
         [6.5294e-01, 2.4075e+00, 2.1283e-01, 8.7660e-03],
         ...,
         [1.0348e+01, 2.4089e-01, 1.3883e-01, 2.4028e-03],
         [8.8505e-01, 2.1575e-01, 1.0223e-01, 1.6869e-03],
         [7.3108e+00, 3.1347e-01, 2.1101e-01, 2.8075e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [8.7575e-01, 4.8737e-01, 2.5008e-01, 1.7486e-03],
         [4.0852e+00, 2.4645e-01, 2.9152e-01, 1.0122e-03],
         ...,
         [3.3662e-01, 2.5399e-01, 8.0090e-02, 1.9290e-03],
         [9.1428e+00, 3.1369e-01, 1.3354e-01, 2.8846e-03],
         [4.3999e


Train Diffusion:   1%|▏         | 20/1500 [00:11<13:36,  1.81it/s]

...


Train Diffusion:  34%|███▍      | 510/1500 [04:51<09:54,  1.67it/s]

Moving average ELBO loss at 510 iterations is: -4748.950927734375. Best ELBO loss value is: -5282.7099609375.

x with CO2 means across time = tensor([[1.9682e+01, 1.1272e-01, 6.3532e-01, 1.0269e-03],
        [1.9631e+01, 1.1761e-01, 6.5103e-01, 1.0551e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.0839e+01, 1.8589e-01, 1.5817e-01, 1.3954e-03],
         [4.1228e+01, 4.9487e-02, 2.0103e-01, 9.8521e-04],
         ...,
         [1.0964e+01, 1.7250e-01, 7.9823e-01, 2.1248e-03],
         [1.2562e+01, 1.2109e-01, 7.1648e-01, 1.8135e-03],
         [1.2525e+01, 1.4859e-01, 6.7473e-01, 2.0062e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.1661e+01, 4.5988e-02, 1.0045e-01, 9.1447e-04],
         [4.1130e+01, 1.4626e-05, 7.8011e-02, 7.8628e-04],
         ...,
         [1.2072e+01, 1.1267e-01, 1.0064e+00, 1.8035e-03],
         [1.2629e+01, 1.4602e-01, 1.0299e+00, 2.0963e-03],
         [1.2202


Train Diffusion:  35%|███▍      | 520/1500 [04:57<10:55,  1.50it/s]

Moving average ELBO loss at 520 iterations is: -4637.344189453125. Best ELBO loss value is: -5282.7099609375.

x with CO2 means across time = tensor([[1.9706e+01, 1.1235e-01, 6.7051e-01, 1.0330e-03],
        [1.9688e+01, 1.1920e-01, 7.0819e-01, 1.0681e-03]],
       grad_fn=<MeanBackward1>)

x with CO2 = tensor([[[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.3404e+01, 9.3965e-02, 1.4667e-01, 1.1191e-03],
         [4.2993e+01, 1.4619e-05, 1.3929e-01, 8.3026e-04],
         ...,
         [1.2509e+01, 1.7381e-01, 1.0583e+00, 2.2973e-03],
         [1.3518e+01, 1.6851e-01, 8.7938e-01, 2.2684e-03],
         [1.3546e+01, 1.5305e-01, 9.1909e-01, 2.1692e-03]],

        [[4.5660e+01, 7.1469e-02, 7.1469e-01, 1.1000e-03],
         [4.3968e+01, 3.7243e-02, 7.9811e-02, 9.2211e-04],
         [4.4220e+01, 4.0137e-02, 1.4510e-01, 9.9960e-04],
         ...,
         [1.2801e+01, 1.1620e-01, 7.9597e-01, 1.8075e-03],
         [1.3999e+01, 9.8795e-02, 1.0072e+00, 1.8138e-03],
         [1.3500


Train Diffusion:  35%|███▍      | 523/1500 [04:59<10:13,  1.59it/s]

In [ ]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'CON_synthetic_sol_df.csv', 0.1, state_dim_SCON, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SCON_SS, SCON_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_CON, get_CO2_CON)

In [ ]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_C, SAWB_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB, get_CO2_AWB)

In [ ]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_SS, SAWB_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB, get_CO2_AWB)

In [ ]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_C, SAWB_ECA_C_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA, get_CO2_AWB_ECA)

In [ ]:
train(devi, l_r, niter, piter, batch_size, ObsModel, csv_to_obs_df, 'AWB_ECA_synthetic_sol_df.csv', 0.1, state_dim_SAWB, t, dt, n, t_span_tensor, i_s_tensor, i_d_tensor, drift_diffusion_SAWB_ECA_SS, SAWB_ECA_SS_params_dict, temp_gen, temp_ref, analytical_steady_state_init_AWB_ECA, get_CO2_AWB_ECA)